In [ ]:
## Watch this to understand how to use the ecosystem daata as quickly as possible
# https: // www.dropbox.com/sh/iysbzchdix2dc55/AAD9t53ntFNtTbyAYyIIdtjRa?dl = 0
# https://www.kaggle.com/c/melbourne-university-seizure-prediction
# Dr Levin Kuhlmann (levin.kuhlmann@monash.edu),

## Imports

In [4]:
import pandas as pd
import scipy.io as sio
import numpy as np
import os
import glob
